# 11411/611 – NLP (S22)
## HW3 – Language Models

**File version:** 1.0.3

Whether it is transcribing spoken utterances as correct word sequences or generating coherent human-like text, language models are extremely useful.

In this assignment, you will be building your own language model powered by n-grams.

### There are two major components in this HW:
#### Part 1: Programming [60 marks]
You are required to program an n-gram language model.

#### Part 2: Analyses [40 marks]
After writing the code, you are required to answer the empirical questions in the handout

### Submission Guidelines

**Deadline:** February 17th, 2022 at 11:59pm EST

**Programming:** 
- This notebook contains helpful test cases and additional information about the programming part of the HW. However, you are only required to submit `lm.py` and `utils.py` on Gradescope.
- We recommended that you first code in the notebook and then copy the corresponding methods/classes to `lm.py`.

**Written:**
- Analyses questions would require you to run your code.
- You will submit the seperate written pdf

## Part 1: Language Models [60 points]

### Step 0: Importing essential libraries

In [97]:
from collections import Counter
from itertools import product
import math
from collections import defaultdict

### Step 1: Preprocessing

We provide you with a few functions in `utils.py` to read and preprocess your input data. Do not edit this file!

In [98]:
from utils import *

We have performed a round of preprocessing on the datasets.

- Each file contains one sentence per line.
- All punctuation marks have been removed.
- Each line is a sequences of tokens separated by whitespace.

#### Special Symbols ( Already defined in `utils.py` )

In [99]:
print("Sentence START symbol: {}".format(START))
print("Sentence END symbol: {}".format(EOS))


Sentence START symbol: <s>
Sentence END symbol: </s>


#### Reading and processing an example file

In [100]:
# Preprocessing example for bigrams (n=2)
sample = read_file("data/sample.txt")
print(sample)

['We are never ever ever ever ever getting back together\n', 'We are the ones together we are back']


In [101]:
# Checkout utils.py to know more about the methods
sample = preprocess(sample, n=2)
print(sample)

[['<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>'], ['<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']]


### Step 2: Helper Functions

In [102]:
def flatten(lst):
    """
    Flattens a nested list into a 1D list.
    Args:
        lst: Nested list (2D)
    
    Returns:
        Flattened 1-D list
    """
    
    return [item for sublist in lst for item in sublist]

In [103]:
print(flatten([["a", "b", "c"], ["d"]]))

['a', 'b', 'c', 'd']


### Step 3: Get started!

### TO DO: `get_ngrams()`

In [129]:
#######################################
# TO-DO: get_ngrams()
#######################################
def get_ngrams(list_of_words, n):
    """
    Returns a list of n-grams for a list of words.
    Args
    ----
    list_of_words: List[str]
        List of already preprocessed and flattened (1D) list of tokens e.g. ["<s>", "hello", "</s>", "<s>", "bye", "</s>"]
    n: int
        n-gram order e.g. 1, 2, 3
    
    Returns:
        n_grams: List[Tuple]
            Returns a list containing n-gram tuples
    """

    list_zip = []
    for i in range(len(list_of_words) - n + 1):
        list_zip.append(tuple(list_of_words[i:i+n]))

    return list_zip

In [130]:
n_grams = get_ngrams(flatten(sample), 3)
assert n_grams == [('<s>', 'we', 'are'), ('we', 'are', 'never'), ('are', 'never', 'ever'), ('never', 'ever', 'ever'), ('ever', 'ever', 'ever'), ('ever', 'ever', 'ever'), ('ever', 'ever', 'getting'), ('ever', 'getting', 'back'), ('getting', 'back', 'together'), ('back', 'together', '</s>'), ('together', '</s>', '<s>'), ('</s>', '<s>', 'we'), ('<s>', 'we', 'are'), ('we', 'are', 'the'), ('are', 'the', 'ones'), ('the', 'ones', 'together'), ('ones', 'together', 'we'), ('together', 'we', 'are'), ('we', 'are', 'back'), ('are', 'back', '</s>')]

### **TO DO:** Class `LanguageModel()`

Now, we will define our LanguageModel class.

**Required parameters:**
- self.model: `dict` of n-grams and their corresponding probabilities.
- self.vocab: `dict` of unigram vocabulary with counts.
- self.n: `int` value for n-gram order (e.g. 1,2,3).
- self.train_data: `List[List]` containing preprocessed train sentences.
- self.alpha: `float` indicates the lazy laplace smoothing parameter (Can theoritically take any non-negative real value, stick to values between 0 and 1 for now).

In `lm.py`, we will be taking most of these argumemts from command line using this command:

`python3 lm.py --train data/sample.txt --test data/sample.txt --n 3 --alpha 0`


In [155]:
#######################################
# TO-DO: LanguageModel()
#######################################
class LanguageModel():
    def __init__(self, n, train_data, alpha=1):
        """
        Language model class.
        
        Args
        ____
        n: int
            n-gram order
        train_data: List[List]
            already preprocessed list of sentences. e.g. [["<s>", "hello", "my", "</s>"], ["<s>", "hi", "there", "</s>"]]
        alpha: float
            Smoothing parameter
        
        Other required parameters:
            self.vocab: vocabulary dict with counts
            self.model: n-gram language model, i.e., n-gram dict with probabilties
            self.n_grams_counts: Frequency count for each of the n-grams present in the training data
            self.prefix_counts: Frequency count of all the corresponding n-1 grams present in the training data
        """
        self.n = n
        self.train_data = train_data
        self.tokens = flatten(self.train_data)
        self.n_grams_counts = None
        self.prefix_counts = None
        self.vocab  = Counter(self.tokens)
        self.alpha = alpha
        self.model = self.build()

    def get_smooth_probabilites(self,n_gram):
        """
        Returns the smoothed probability of a single ngram, using Laplace Smoothing. Remember to handle the special case of n=1
        Use the class variables we defined in the build function. It is suggested to implement the build function before this one.
        """
        if self.n == 1:
            prob = (self.n_grams_counts[n_gram]+self.alpha)  /   (len(self.tokens)+self.alpha*len(self.vocab))
        else:
            denominator = self.prefix_counts[n_gram[:-1]]
            prob = (self.n_grams_counts[n_gram]+self.alpha)/(denominator+self.alpha*len(self.vocab))
        
        return prob
    
    
    #TODO:
    def build(self):
        """
        Returns a n-gram (could be a unigram) dict with n-gram tuples as keys and probabilities as values. 
        It could be a unigram model as well
        """
        
        # TODO: Get the n-grams from the training data using the previously defined methods
        n_grams = get_ngrams(self.tokens, self.n)
        n_1_grams = get_ngrams(self.tokens, self.n - 1)
        # TODO: Define the class variables n_grams_counts and prefix_counts 
        self.n_grams_counts = Counter(n_grams)
        self.prefix_counts = Counter(n_1_grams)

        # TODO Get the Probabilities using the get_smooth_probabilities
        model = {}
        for item, count in self.n_grams_counts.items():
            prob = self.get_smooth_probabilites(item)
            model[item] = prob

        return model

In [156]:
# Quick test
test_lm = LanguageModel(n=2, train_data=sample, alpha=0)

In [157]:
assert test_lm.vocab == Counter({'<s>': 2,'we': 3,'are': 3,'never': 1,'ever': 4,'getting': 1,'back': 2,'together': 2,'</s>': 2,'the': 1,'ones': 1})

In [158]:
assert test_lm.model =={('<s>', 'we'): 1.0, ('we', 'are'): 1.0, ('are', 'never'): 0.3333333333333333, ('never', 'ever'): 1.0,('ever', 'ever'): 0.75, ('ever', 'getting'): 0.25,('getting', 'back'): 1.0,('back', 'together'): 0.5,('together', '</s>'): 0.5,('</s>', '<s>'): 0.5, ('are', 'the'): 0.3333333333333333,('the', 'ones'): 1.0,('ones', 'together'): 1.0,('together', 'we'): 0.5,('are', 'back'): 0.3333333333333333, ('back', '</s>'): 0.5}

### **TO DO:**  `perplexity()`

Steps:
1. First, create n-grams after flattening the data.
2. If a matching n-gram is not found, perform Lazy Laplace Smoothing. Remember to memoize the new n-gram probabilities to speed up calculation
3. Refer to the perplexity equation in the language model chapter of the textbook required for this course. [[Link to the chapter]](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

Tips:
- Remember that product changes summation under `log`. Take log of probabilities (`math.log()`), sum them up (`sum()`) and then exponentiate it (`math.exp()`) to get back to the original scale.
- Make sure to `flatten()` your data before creating the n_grams using `get_ngrams().`

In [325]:
#######################################
# TO-DO: perplexity()
#######################################
def perplexity(lm, test_data):
    """
    Returns perplexity calculated on the test data.
    Args
    ----------
    test_data: List[List] 
        Already preprocessed nested list of sentences
        
    lm: LanguageModel class object
        To be used for retrieving lm.model, lm.n and lm.vocab

    Returns
    -------
    float
        Calculated perplexity value
    """
    #math.exp(sum(math.log()))
    # TODO Flatten and get the n-grams
    n_grams = get_ngrams(flatten(test_data), lm.n)
    
    # TODO Calculate the Perplexity over all the test n-grams
    probs = lm.model
    log_sum = 0
    for item in n_grams:
        try:
            if probs[item] != 0:
                log_sum += math.log(probs[item])
        except:
            if lm.n == 1:
                prob = (lm.alpha)/(len(lm.tokens)+lm.alpha*len(lm.vocab))
            else:
                denominator = lm.prefix_counts[item[:-1]]
                prob = (lm.alpha)/(denominator+lm.alpha*len(lm.vocab))
            
            log_sum += math.log(prob)
        
    perplexity = math.exp(-log_sum/len(n_grams))
    return perplexity

In [326]:
# Quick test
test_lm = LanguageModel(n=2, train_data=sample, alpha=0)
test_ppl = perplexity(test_lm, sample)
print(test_ppl)
assert test_ppl < 1.7

1.5358609630414888


### Step 4: Bringing everything together!

**Note:** Most of these will already be defined for you in `main()` method in `lm.py`

In [327]:
# Arguments

train_path = "data/bbc/business.txt"
test_path = "data/bbc/business.txt"
n = 3
min_freq = 1
smoothing = 0.1

In [328]:
train = read_file(train_path)
test = read_file(test_path)

In [329]:
print("No of sentences in train file: {}".format(len(train)))
print("No of sentences in test file: {}".format(len(test)))

No of sentences in train file: 19990
No of sentences in test file: 19990


In [330]:
print("Raw train example: \n{}".format(train[2]))
print("Raw test example: \n{}".format(test[2]))

Raw train example: 
13bn £ 600m for the three months to December from 639m year earlier

Raw test example: 
13bn £ 600m for the three months to December from 639m year earlier



In [331]:
# Basic preprocessing
train = preprocess(train, n)
test = preprocess(test, n)

In [332]:
print("Preprocessed train example: \n{}\n".format(train[2]))
print("Preprocessed test example: \n{}".format(test[2]))

Preprocessed train example: 
['<s>', '<s>', '13bn', '£', '600m', 'for', 'the', 'three', 'months', 'to', 'december', 'from', '639m', 'year', 'earlier', '</s>']

Preprocessed test example: 
['<s>', '<s>', '13bn', '£', '600m', 'for', 'the', 'three', 'months', 'to', 'december', 'from', '639m', 'year', 'earlier', '</s>']


In [333]:
print("Loading {}-gram model.".format(n))
lm = LanguageModel(n, train, smoothing)
print("Vocabulary size (unique unigrams): {}".format(len(lm.vocab)))
print("Total number of unique n-grams: {}".format(len(lm.model)))

Loading 3-gram model.
Vocabulary size (unique unigrams): 11916
Total number of unique n-grams: 141221


In [334]:
# Calculate perplexity

ppl = perplexity(lm, test)
print("Model perplexity: {:.3f}".format(ppl))

Model perplexity: 262.742


### Step 5: Experimenting with generations!

**Note**: These methods are already written for you. Use them to solve Written questions.

In [335]:
import random

random.seed(11411)

In [336]:
def best_candidate(lm, prev, i, without=[], mode="random"):
    """
    Returns the most probable word candidate after a given sentence.
    """
    blacklist  = ["<UNK>"] + without
    candidates = ((ngram[-1],prob) for ngram,prob in lm.model.items() if ngram[:-1]==prev)
    candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
    candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
    if len(candidates) == 0:
        return ("</s>", 1)
    else:
        if(mode=="random"):
            return candidates[random.randrange(len(candidates))]
        else:
            return candidates[0]

In [337]:
def top_k_best_candidates(lm, prev, k, without=[]):
    """
    Returns the K most-probable word candidate after a given n-1 gram.
    Args
    ----
    lm: LanguageModel class object
    
    prev: n-1 gram
        List of tokens n
    """
    blacklist  = ["<UNK>"] + without
    candidates = ((ngram[-1],prob) for ngram,prob in lm.model.items() if ngram[:-1]==prev)
    candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
    candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
    if len(candidates) == 0:
        return ("</s>", 1)
    else:
        return candidates[:k]

In [338]:
def generate_sentences_from_phrase(lm, num, sent, prob, mode):
    """
    Generate sentences using the trained language model.
    """
    min_len=12
    max_len=24
    
    for i in range(num):
        while sent[-1] != "</s>":
            prev = () if lm.n == 1 else tuple(sent[-(lm.n-1):])
            blacklist = sent + (["</s>"] if len(sent) < min_len else [])

            next_token, next_prob = best_candidate(lm, prev, i, without=blacklist, mode=mode)
            sent.append(next_token)
            prob *= next_prob
            
            if len(sent) >= max_len:
                sent.append("</s>")

        yield ' '.join(sent), -1/math.log(prob)

In [339]:
lm.n_grams_counts.most_common()

[(('</s>', '<s>', '<s>'), 19989),
 (('<s>', '<s>', 'the'), 3474),
 (('said', '</s>', '<s>'), 688),
 (('<s>', '<s>', 'it'), 640),
 (('1', '</s>', '<s>'), 560),
 (('<s>', '<s>', 'in'), 528),
 (('<s>', '<s>', 'but'), 516),
 (('<s>', '<s>', 'mr'), 422),
 (('<s>', '<s>', 'however'), 380),
 (('year', '</s>', '<s>'), 346),
 (('<s>', '<s>', 'a'), 336),
 (('2', '</s>', '<s>'), 322),
 (('<s>', '<s>', 'we'), 292),
 (('0', '</s>', '<s>'), 248),
 (('4', '</s>', '<s>'), 234),
 (('<s>', '<s>', 'he'), 222),
 (('the', 'world', 's'), 218),
 (('3', '</s>', '<s>'), 210),
 (('<s>', '<s>', '5'), 204),
 (('<s>', '<s>', 'this'), 196),
 (('the', 'country', 's'), 192),
 (('in', 'the', 'us'), 184),
 (('<s>', '<s>', '1'), 176),
 (('5', '</s>', '<s>'), 174),
 (('he', 'said', '</s>'), 172),
 (('<s>', '<s>', 'us'), 172),
 (('<s>', 'the', 'company'), 166),
 (('<s>', '<s>', 'analysts'), 166),
 (('2004', '</s>', '<s>'), 164),
 (('<s>', '<s>', '3'), 160),
 (('years', '</s>', '<s>'), 154),
 (('<s>', 'the', 'us'), 150),
 

## Part 2: Written [40 points]. We have given some code for some of the written parts to make it easier for you.

### **Written 3.3** – Song Attribution [8 points]

In [343]:
# Example code for Taylor Swift LM
n = 3
smoothing = 0.1
min_freq = 1

train = read_file("data/lyrics/green_day.txt")
test = read_file("data/lyrics/test_lyrics.txt")

train = preprocess(train, n)
test = preprocess(test, n)
lm = LanguageModel(n, train, smoothing)

ppl = perplexity(lm, test)
print(ppl)

522.5401188730924


### **Written 3.4.1** –  Intro to Decoding [8 points]

In [344]:
n = 3
smoothing = 0.1
min_freq = 1

In [357]:
train = read_file("data/bbc/entertainment.txt")
train = preprocess(train, n)

lm = LanguageModel(n, train, smoothing)

In [346]:
s1 = ("number", "three")

s2 = ("starred", "in")

s3 = ("actor", "in")

In [348]:
# TODO Get the top 5 candidates for next best word
top_k_best_candidates(lm, s3, 5, without=['<s>', '</s>'])

[('a', 0.005177389237820403), ('2004', 0.0009336275674758106)]

### **Written 3.4.2** – Text Generation [8 points]

For this subtask, use the LM trained in Written 3.4.1

Selecting words sequentially from a probability distribution is called _decoding_.

Two popular decoding approaches are,
1. **Max-probability decoding** - We consistently choose the candidate with maximum probability.
2. **Random Sampling** - We sample a candidate randomly.
2. **top-K Sampling** - We sample a candidate randomly from the top-K most probable choices.

In this part, we will try the first two approaches to generate sentences.

Q1. Use `generate_sentences()` method to generate sentences after the provided phrases from `s1` to `s3`. Use modes `random` and `max`. Report one of your favourite generations (for any strategy or phrase). Which mode you think is better and why?



In [359]:
# Random
for _ in range(5):
    print(list(generate_sentences_from_phrase(lm, 1, ["<s>", "<s>", "number", "three"], 0.2, mode="max")))

[('<s> <s> number three during its first week on home entertainment release the film s director richard eyre issued a warning earlier in his </s>', 0.007691174546401956)]
[('<s> <s> number three during its first week on home entertainment release the film s director richard eyre issued a warning earlier in his </s>', 0.007691174546401956)]
[('<s> <s> number three during its first week on home entertainment release the film s director richard eyre issued a warning earlier in his </s>', 0.007691174546401956)]
[('<s> <s> number three during its first week on home entertainment release the film s director richard eyre issued a warning earlier in his </s>', 0.007691174546401956)]
[('<s> <s> number three during its first week on home entertainment release the film s director richard eyre issued a warning earlier in his </s>', 0.007691174546401956)]


**Aside (for fun!)**: Train your LM on Taylor Swift lyrics and generate the next hit!

### **Written 3.5** – Battle of the LMs: GPT-2 vs Trigram [8 points]

For this subtask, you will be generating text and comparing GPT-2 with your n-gram language model. 

Generative pretrained transformer (GPT) is a neural language model series created by OpenAI. The n-gram language model you trained has on average around 10K-20K parameters (`len(lm.model)`.) Compare that to the 175 billion parameters of the latest version of GPT-3!

Let's see how GPT-2 compares to the LM you trained in subtask 4 on `data/bbc/tech-small.txt` dataset.

In [358]:
test = read_file("data/bbc/tech-small.txt")
test = preprocess(test, n)
lm = LanguageModel(n, train, smoothing)
ppl = perplexity(lm, test)
print(ppl)
#TODO: Calculate your n-gram model's perplexity


4889.176510567337


### Computing GPT-2's perplexity on test set

You need to enable a GPU runtime from `Runtime` menu option. Go to `Runtime` → `Change Runtime Type` → `Hardware Accelerator (GPU)`

In [180]:
!pip install transformers
!pip install torch
!pip install tqdm

In [351]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch

model_id = "distilgpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [352]:
test = read_file("data/bbc/tech-small.txt")
encodings = tokenizer("\n\n".join(test), return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (1137 > 1024). Running this sequence through the model will result in indexing errors


In [353]:
from tqdm import tqdm

max_length = model.config.n_positions
stride = 100

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

100%|██████████| 12/12 [00:18<00:00,  1.56s/it]


In [355]:
print("Perplexity using GPT2:", ppl.item())

Perplexity using GPT2: 50.644840240478516
